In [1]:
from IPython.display import display, Markdown
def md(s):
    display(Markdown(s))

import numpy as np

API_URL = "http://localhost:11434/api/generate"

SYSTEM_PROMPT = """You are a helpful AI assistant that can call the function when asked about gear calculations. When given a query about gear trains, respond with:

Function call: toothnumbers({'m': <module>, 'N1': <teeth_on_driving_gear>, 'gearratio': <target_gear_ratio>})

For example, for "module 5 mm, 20 teeth on driving gear, gear ratio 2", respond:

Function call: toothnumbers({'m': 5, 'N1': 20, 'gearratio': 2})

Do not compute the result yourself; let the system handle it. For other queries, respond normally.

Also, whatever the response is, put it all in the LLM response column, not outside of it.

"""

def getinput(prompt):
    try:
        return input(prompt)
    except EOFError:
        return None
    except KeyboardInterrupt:  # When ESC is pressed
        return None

In [2]:
def display_conversation_history(messages):
    """
    Displays conversation history in a two-column markdown table in a Jupyter notebook.
    
    Args:
        messages: List of strings alternating between user queries and LLM responses
    """
    from IPython.display import clear_output
    # Clear the output to avoid clutter
    clear_output(wait=True)
    # Create table header
    table = "| User Query | LLM Response |\n"
    table += "|------------|-------------|\n"
    
    # Add rows to the table, processing messages in pairs
    for i in range(0, len(messages), 2):
        user_msg = messages[i]
        
        # Check if there's a corresponding LLM response
        llm_msg = messages[i+1] if i+1 < len(messages) else ""
        
        # Add row to table, escaping any pipe characters in the messages
        user_msg_escaped = user_msg.replace("|", "\\|")
        llm_msg_escaped = llm_msg.replace("|", "\\|")
        
        table += f"| {user_msg_escaped} | {llm_msg_escaped} |\n"
    
    # Display the table using the md() function
    md(table)

In [3]:

def toothnumbers(m, N1, gearratio):
    """
    Calculate the diameters of the gears in a gear train.

    Parameters:
    m (float): The module of the gears (mm)
    N1 (int): The number of teeth on the driving gear.
    gearratio (float): The target gear ratio.

    Returns:
    dict: A dictionary containing gear details and the actual gear ratio.
    """
    d1 = m * N1  # Diameter of the driving gear
    N2 = int(N1 * gearratio)  # Number of teeth on the driven gear
    d2 = m * N2  # Diameter of the driven gear
    actual_gear_ratio = N2 / N1  # Actual gear ratio

    if abs(actual_gear_ratio - gearratio) > 0.01:
        raise ValueError(f"Gear ratio mismatch: {actual_gear_ratio:.2f} != {gearratio:.2f}")

    return {
        "text": "Computation successful",
        "number_of_teeth_gear1": N1,
        "diameter_gear1_mm": d1,
        "number_of_teeth_gear2": N2,
        "diameter_gear2_mm": d2,
        "actual_gear_ratio": actual_gear_ratio
    }

In [4]:
import requests

def generate_response(prompt):
    full_prompt = f"{SYSTEM_PROMPT}\n\nUser: {prompt}\nAssistant:"
    
    try:
        response = requests.post(
            API_URL,
            json={
                "model": "llama3.2:3b",
                "prompt": full_prompt,
                "stream": False
            },
            timeout=60  # Increased timeout to 60 seconds
        )
        response.raise_for_status()
        result = response.json()
        return result.get("response", "No response")
    except requests.Timeout:
        return "Error: Request timed out. The model is taking too long to respond. Please try again."
    except requests.ConnectionError:
        return "Error: Could not connect to the Ollama server. Please ensure it is running."
    except requests.RequestException as e:
        return f"Error connecting to Llama model: {str(e)}"

In [5]:
import re
import ast

CHAT_HISTORY= []

while True:
    user_input = getinput("Enter a prompt (or 'q' to quit): ")
    if user_input == "1":
        user_input = "Calculate the diameters of the gears in a gear train. The module of the gears is 5 mm, the number of teeth on the driving gear is 20, and the target gear ratio is 2."
    if user_input is None:
        break
    if user_input.lower() == 'q':
        break

    CHAT_HISTORY.append(user_input)
    response = generate_response(user_input)

    # Look for function call pattern
    pattern = r"Function call: toothnumbers\((.*?)\)"
    match = re.search(pattern, response)

    if match:
        args_str = match.group(1)  # e.g., "{'m': 5, 'N1': 20, 'gearratio': 2}"
        try:
            # Parse the arguments into a dictionary
            args_dict = ast.literal_eval(args_str)
            # Call the function
            result = toothnumbers(**args_dict)
            # Format the response
            response = f"""{result['text']}: Driver Gear: {result['number_of_teeth_gear1']} teeth, {result['diameter_gear1_mm']:.1f}mm diameter, Driven Gear: {result['number_of_teeth_gear2']} teeth, {result['diameter_gear2_mm']:.1f}mm diameter, Actual Gear Ratio: {result['actual_gear_ratio']:.3f}"""
            CHAT_HISTORY.append(response)1
            
            print(response)
        except Exception as e:
            error_msg = f"Error in calculation: {str(e)}"
            CHAT_HISTORY.append(error_msg)
            print(error_msg)
    else:
        # No function call, use the raw response
        CHAT_HISTORY.append(response)
        print(response)

    display_conversation_history(CHAT_HISTORY)  
    print(response)

| User Query | LLM Response |
|------------|-------------|
| Calculate the diameters of the gears in a gear train. The module of the gears is 5 mm, the number of teeth on the driving gear is 20, and the target gear ratio is 2. | Computation successful: Driver Gear: 20 teeth, 100.0mm diameter, Driven Gear: 40 teeth, 200.0mm diameter, Actual Gear Ratio: 2.000 |


Computation successful: Driver Gear: 20 teeth, 100.0mm diameter, Driven Gear: 40 teeth, 200.0mm diameter, Actual Gear Ratio: 2.000
